# Using the V3IO Frames Library for High-Performance Data Access 

- [Overview](#frames-overview)
- [Initialization](#frames-init)
- [Working with NoSQL Tables (kv Backend)](#frames-kv)
- [Working with Time-Series Databases (tsdb Backend)](#frames-tsdb)
- [Cleanup](#frames-cleanup)

<a id="frames-overview"></a>
## Overview

[V3IO Frames](https://github.com/v3io/frames) (**"Frames"**) is a multi-model open-source data-access library, developed by Iguazio, which provides a unified high-performance DataFrame API for working with data in the data store of the Iguazio Data Science Platform (**"the platform"**).
Frames currently supports the NoSQL (key-value) and time-series (TSDB) data models via its `nosql`|`kv` and `tsdb` backends.
> **Note:** You can replace any reference to the `kv` backend in this tutorial with the `nosql` alias.

To use Frames, you first need to import the **v3io_frames** library and create and initialize a client object &mdash; an instance of the`Client` class.<br>
The `Client` class features the following object methods for supporting basic data operations; the type of data is derived from the backend type (`kv` &mdash; NoSQL table / `tsdb` &mdash; TSDB table):

- `create` &mdash; creates a new TSDB table ("backend data").
- `delete` &mdash; deletes a table.
- `read` &mdash; reads data from a table into pandas DataFrames.
- `write` &mdash; writes data from pandas DataFrames to a table.
- `execute` &mdash; executes a command on a table.
  Each backend may support multiple commands.

For a detailed description of the Frames API, see the [Frames API reference](https://www.iguazio.com/docs/latest-release/reference/api-reference/frames/).<br>
For more help and usage details, use the internal API help &mdash; `<client object>.<command>?` in Jupyter Notebook or `print(<client object>.<command>.__doc__)`.<br>
For example, the following command returns information about the read operation for a client object named `client`:
```
client.read?
```

<a id="frames-init"></a>
## Initialization

To use V3IO Frames, first ensure that your platform tenant has a shared tenant-wide instance of the V3IO Frames service.
This can be done by a platform service administrator from the **Services** dashboard page.<br>
Then, import the required libraries and create a Frames client object (an instance of the `Client` class), as demonstrated in the following code, which creates a client object named `client`.

> **Note:**
> - The client constructor's `container` parameter is set to `"users"` for accessing data in the platform's "users" data container.
> - Because no authentication credentials are passed to the constructor, Frames will use the access key that's assigned to the `V3IO_ACCESS_KEY` environment variable.
>   The platform's Jupyter Notebook service defines this variable automatically and initializes it to a valid access key for the running user of the service.
>   You can pass different credentials by using the constructor's `token` parameter (platform access key) or `user` and `password` parameters (platform username and password).

In [1]:
import pandas as pd
import v3io_frames as v3f
import os

# Create a Frames client
client = v3f.Client("framesd:8081", container="users")

<a id='frames-kv'></a>
## Working with NoSQL Tables (kv Backend)

This section demonstrates how to use the `kv` Frames backend to write and read NoSQL data in the platform.

- [Initialization](#frames-kv-init)
- [Write to a NoSQL Table](#frames-kv-write)
- [Read from the Table Using an SQL Query](#frames-kv-read-sql-query)
- [Read from the Table Using the Frames API](#frames-kv-read-frames-api)
  - [Read Using a Single DataFrame](#frames-kv-read-frames-api-single-df)
  - [Read Using a DataFrames Iterator (Streaming)](#frames-kv-read-frames-api-df-iterator)
- [Delete the NoSQL Table](#frames-kv-delete)

<a id="frames-kv-init"></a>
### Initialization

Start out by defining table-path variables that will be used in the tutorial's code examples.<br>
The table path (`table`) is relative to the configured parent data container; see [Write to a NoSQL Table](#frames-kv-write).

In [2]:
# Relative path to the NoSQL table within the parent platform data container
table = os.path.join(os.getenv("V3IO_USERNAME"), "examples", "bank")

# Full path to the NoSQL table for SQL queries (platform Presto data-path syntax);
# use the same data container as used for the Frames client ("users")
sql_table_path = 'v3io.users."' + table + '"'

<a id="frames-kv-write"></a>
### Write to a NoSQL Table

Read a file from an Amazon Simple Storage (S3) bucket into a Frames pandas DataFrame, and use the `write` method of the Frames client with the `kv` backend to write the data to a NoSQL table.<br>
The mandatory `table` parameter specifies the relative table path within the data container that was configured for the Frames client (see the [main initialization](#frames-init) step).
In the following example, the relative table path is set by using the `table` variable that was defined in the [kv backend initialization](#frames-kv-init) step.<br>
The `dfs` parameter can be set either to a single DataFrame (as done in the following example) or to multiple DataFrames &mdash; either as a DataFrames iterator or as a list of DataFrames.

In [3]:
# Prepare the ingestion data by reading an AWS S3 file into a DataFrame
df = pd.read_csv("https://s3.amazonaws.com/iguazio-sample-data/bank.csv", sep=";")
# Display DataFrame info & head (optional - for testing)
# display(df.info(), df.head())

In [4]:
# Write data from a DataFrame to a NoSQL table
client.write("kv", table=table, dfs=df)

<a id="frames-kv-read-sql-query"></a>
### Read from the Table Using an SQL Query

You can run SQL queries on your NoSQL table (using Presto) to offload data filtering, grouping, joins, etc. to a scale-out high-speed database engine.

> **Note:** To query a table in a platform data container, the table path in the `from` section of the SQL query should be of the format `v3io.<container name>."/path/to/table"`.
> See [Presto Data Paths](https://www.iguazio.com/docs/latest-release/tutorials/getting-started/fundamentals/#data-paths-presto) in the platform documentation.
> In the following example, the path is set by using the `sql_table_path` variable that was defined in the [kv backend initialization](#frames-kv-init) step.
> Unless you changed the code, this variable translates to `v3io.users."<running user>/examples/bank"`; for example, `v3io.users."iguazio/examples/bank"` for user "iguazio".

In [5]:
%sql select * from $sql_table_path where balance > 10000 limit 8

Done.


loan,education,previous,housing,poutcome,duration,marital,default,balance,month,contact,campaign,y,idx,job,day,age,pdays
no,unknown,0,yes,unknown,117,divorced,no,10287,may,unknown,1,no,899,blue-collar,29,51,-1
yes,tertiary,0,yes,unknown,197,divorced,no,13204,nov,cellular,2,no,3329,management,20,34,-1
no,secondary,0,yes,unknown,163,married,no,10888,aug,cellular,1,no,4346,technician,5,44,-1
no,secondary,0,no,unknown,5,married,no,10655,jul,telephone,3,no,3967,technician,31,48,-1
yes,secondary,0,no,unknown,77,married,no,13229,jul,telephone,2,no,3213,admin.,8,49,-1
no,tertiary,3,yes,success,638,single,no,13711,may,cellular,1,no,1779,technician,14,32,175
no,primary,4,yes,success,146,married,no,12519,apr,cellular,2,no,602,blue-collar,17,50,147
no,secondary,0,yes,unknown,135,divorced,no,10787,jun,unknown,2,no,3345,technician,4,31,-1


<a id="frames-kv-read-frames-api"></a>
### Read from the Table Using the Frames API

Use the `read` method of the Frames client with the `kv` backend to read data from your NoSQL table.<br>
The `read` method can return a DataFrame or a DataFrames iterator (a stream), as demonstrated in the following examples.

- [Read Using a Single DataFrame](#frames-kv-read-frames-api-single-df)
- [Read Using a DataFrames Iterator (Streaming)](#frames-kv-read-frames-api-df-iterator)

<a id="frames-kv-read-frames-api-single-df"></a>
#### Read Using a Single DataFrame

The following example uses a single command to read data from the NoSQL table into a DataFrame.

In [6]:
df = client.read(backend="kv", table=table, filter="balance > 20000")
df.head(8)

,age,balance,campaign,contact,day,default,duration,education,housing,job,loan,marital,month,pdays,poutcome,previous,y
idx,,,,,,,,,,,,,,,,,
3700,60,71188,1,cellular,6,no,205,primary,no,retired,no,married,oct,-1,unknown,0,no
3332,31,22546,6,cellular,14,no,8,tertiary,yes,management,no,married,may,267,failure,4,no
2624,53,22370,1,unknown,15,no,106,tertiary,yes,entrepreneur,no,married,may,-1,unknown,0,no
2776,37,22856,1,cellular,2,no,154,primary,no,management,no,married,jul,388,failure,1,no
1483,43,27733,7,unknown,3,no,164,tertiary,yes,technician,no,single,jun,-1,unknown,0,no
650,33,23663,2,cellular,16,no,199,tertiary,yes,housemaid,no,single,apr,146,failure,2,no
4334,37,20453,1,telephone,4,no,115,secondary,yes,entrepreneur,no,single,may,-1,unknown,0,no
1031,49,25824,1,unknown,17,no,94,primary,no,retired,no,single,jun,-1,unknown,0,no


<a id="frames-kv-read-frames-api-df-iterator"></a>
#### Read Using a DataFrames Iterator (Streaming)

The following example uses a DataFrames iterator to stream data from the NoSQL table into multiple DataFrames and allow concurrent data movement and processing.<br>
The example sets the `iterator` parameter to `True` to receive a DataFrames iterator (instead of the default single DataFrame), and then iterates the DataFrames in the returned iterator; you can also use `concat` instead of iterating the DataFrames.

> **Note:** Iterators work with all Frames backends and can be used as input to write functions that support this, such as the `write` method of the Frames client.

In [7]:
dfs = client.read(backend="kv", table=table, filter="balance > 20000",
                  iterator=True)
for df in dfs:
    display(df.head())

,age,balance,campaign,contact,day,default,duration,education,housing,job,loan,marital,month,pdays,poutcome,previous,y
idx,,,,,,,,,,,,,,,,,
3700,60,71188,1,cellular,6,no,205,primary,no,retired,no,married,oct,-1,unknown,0,no
3332,31,22546,6,cellular,14,no,8,tertiary,yes,management,no,married,may,267,failure,4,no
1483,43,27733,7,unknown,3,no,164,tertiary,yes,technician,no,single,jun,-1,unknown,0,no
650,33,23663,2,cellular,16,no,199,tertiary,yes,housemaid,no,single,apr,146,failure,2,no
4334,37,20453,1,telephone,4,no,115,secondary,yes,entrepreneur,no,single,may,-1,unknown,0,no


<a id="frames-kv-delete"></a>
### Delete the NoSQL Table

Use the `delete` method of the Frames client with the `kv` backend to delete the NoSQL table that was used in the previous steps.

In [8]:
# Delete the `table` NoSQL table
client.delete("kv", table)

<a id='frames-tsdb'></a>
## Working with Time-Series Databases (tsdb Backend)

This section demonstrates how to use the `tsdb` Frames backend to create a time-series database (TSDB) table in the platform, ingest data into the table, and read from the table (i.e., submit TSDB queries).

- [Initialization](#frames-tsdb-init)
- [Create a TSDB Table](#frames-tsdb-create)
- [Write to the TSDB Table](#frames-tsdb-write)
- [Read from the TSDB Table](#frames-tsdb-read)
  - [Conditional Read](#frames-tsdb-read-conditional)
- [Delete the TSDB Table](#frames-tsdb-delete)

<a id="frames-tsdb-init"></a>
### Initialization

Start out by defining a TSDB table-path variable that will be used in the tutorial's code examples.<br>
The table path (`tsdb_table`) is relative to the configured parent data container; see [Create a TSDB Table](#frames-tsdb-create).

In [9]:
# Relative path to the TSDB table within the parent platform data container
tsdb_table = os.path.join(os.getenv("V3IO_USERNAME"), "examples", "tsdb_tab")

<a id="frames-tsdb-create"></a>
### Create a TSDB Table

Use the `create` method of the Frames client with the `tsdb` backend to create a new TSDB table.<br>
The mandatory `table` parameter specifies the relative table path within the data container that was configured for the Frames client (see the [main initialization](#frames-init) step).
In the following example, the relative table path is set by using the `tsdb_table` variable that was defined in the [tsdb backend initialization](#frames-tsdb-init) step.<br>
You must set the `rate` argument to the ingestion rate of the TSDB metric-samples, as `"[0-9]+/[smh]"` (where '`s`' = seconds, '`m`' = minutes, and '`h`' = hours); for example, `1/s` (one sample per minute).
It's recommended that you set the rate to the average expected ingestion rate, and that the ingestion rates for a given TSDB table don't vary significantly; when there's a big difference in the ingestion rates (for example, x10), use separate TSDB tables.
You can also set additional optional arguments, such as `aggregates` or `aggregation_granularity`.

In [10]:
# Create a new TSDB table; ingestion rate = one sample per hour ("1/h")
client.create(backend="tsdb", table=tsdb_table, rate="1/h")

<a id="frames-tsdb-write"></a>
### Write to the TSDB Table

Use the `write` method of the Frames client with the `tsdb` backend to ingest data from a pandas DataFrame into your TSDB table.<br>
The primary-key attribute of platform TSDB tables (i.e., the DataFrame index column) must hold the sample time of the data (displayed as `time` in read outputs).<br>
In addition, TSDB table items (rows) can optionally have sub-index columns (attributes) that are called labels.
You can add labels to TSDB table items in one of two ways; you can also combine these methods:

- Use the `labels` dictionary parameter of the `write` method to add labels to all the written metric-sample table items (DataFrame rows) &mdash; `{<label>: <value>[, <label>: <value>, ...]}`.<br>
  For example, `{"node": "11", "os": "linux"}`.
  Note that the label values must be provided as strings.
- Define DataFrame index columns for the labels.
  All DataFrame index columns except for the sample-time index column are automatically converted into labels for the respective table items.
  > **Note:** If you wish to use regular columns in your DataFrames as metric labels, convert these columns to index columns.
  > The following example converts the `symbol` and `exchange` columns to index columns that will be used as metric labels (in addition to the `time` index column):<br>
  > ```python
  > df.index.name="time"                              # Name the sample-time index column "time"
  > df.reset_index(level=0, inplace=True)             # Reset the DataFrame indexes
  > df = df.set_index(["time", "symbol", "exchange"]) # Define the time and label columns as index columns
  > ```

In [11]:
import numpy as np
from datetime import datetime, timedelta


# Generate a DataFrame with TSDB metric samples and a "time" index column
def gen_df_w_tsdb_data(num_items=24, freq="1H", end=None, start=None,
                       start_delta=None, tz=None, normalize=False, zero=False,
                       attrs=["cpu", "mem", "disk"]):
    if (start is None and start_delta is not None and end is not None):
        start = end - timedelta(days=start_delta)
    if (zero):
        if (end is not None):
            end = end.replace(minute=0, second=0, microsecond=0)
        if (start is not None):
            start = start.replace(minute=0, second=0, microsecond=0)
    # If `start`, `end`, `num_items` (date_range() `periods`), and `freq`
    # are set, ignore `freq`
    if (freq is not None and start is not None and end is not None and
            num_items is not None):
        freq = None
    times = pd.date_range(periods=num_items, freq=freq, start=start, end=end,
                          tz=tz, normalize=normalize)
    data = np.random.rand(num_items, len(attrs)) * 100
    df = pd.DataFrame(data, index=times, columns=attrs)
    df.index.name = "time"
    return df

In [12]:
# Prepare DataFrames with randomly generated metric samples
end_t = datetime.now()
start_delta = 7  # start time = ent_t - 7 days
dfs = []
for i in range(4):
    # Generate a new DataFrame with TSDB metrics
    dfs.append(gen_df_w_tsdb_data(end=end_t, start_delta=7, zero=True))
    # Display DataFrame info & head (optional - for testing)
    # print("\n** dfs[" + str(i) + "] **")
    # display(dfs[i].info(), dfs[i].head())

In [13]:
# Write to a TSDB table

# Prepare metric labels to write
labels = [
    {"node": "11", "os": "linux"},
    {"node": "2", "os": "windows"},
    {"node": "11", "os": "windows"},
    {"node": "2", "os": "linux"}
]

# Write the contents of the prepared DataFrames to a TSDB table. Use multiple
# write commands with the `labels` parameter to set different label values.
num_dfs = len(dfs)
for i in range(num_dfs):
    client.write("tsdb", table=tsdb_table, dfs=dfs[i], labels=labels[i])

<a id="frames-tsdb-read"></a>
### Read from the TSDB Table

- [Overview and Basic Examples](#frames-tsdb-read-basic)
- [Conditional Read](#frames-tsdb-read-conditional)

<a id="frames-tsdb-read-basic"></a>
#### Overview and Basic Examples

Use the `read` method of the Frames client with the `tsdb` backend to read data from your TSDB table (i.e., query the database).<br>
Note that you cannot mix raw sample-data queries and aggregation queries.

You must set the `table` parameter to the path to the TSDB table.<br>
You can optionally set additional method parameters to configure the query:

- `columns` defines the query metrics (default = all).
- `aggregators` defines aggregation functions ("aggregators") to execute for all the configured metrics.
- `filter` restricts the query by using a platform [filter expression](https://www.iguazio.com/docs/latest-release/reference/expressions/condition-expression/#filter-expression).
- `start` and `end` define the query's time range &mdash; the metric-sample timestamps to which to apply the query.
   The default `end` time is `"now"` and the default `start` time is 1 hour before the end time (`<end> - 1h`).
- `step` defines the interval for aggregation or raw-data downsampling (default = the query's time range).
- `multi_index` casn be set to `True` to return labels as index columns, as demonstrated in the following examples.
  By default, only the metric sample-time primary-key attribute is returned as an index column.

See the [Frames API reference](https://www.iguazio.com/docs/latest-release/reference/api-reference/frames/tsdb/read/) for more information about the `read` parameters that are supported for the `tsdb` backend.

In [14]:
# Read all metrics from the TSDB table (start="0"; default `end` time = "now")
# into a single DataFrame (default `Iterator`=False) and display the first 10
# items; show metric labels as index columns (multi_index=True)
df = client.read(backend="tsdb", table=tsdb_table, start="0", multi_index=True)
display(df.head(8))

,,,cpu,disk,mem
time,node,os,,,
2020-03-22 20:00:00+00:00,11,linux,71.245621,91.954313,20.934462
2020-03-23 03:18:15.652000+00:00,11,linux,3.868040,69.360057,23.388007
2020-03-23 10:36:31.304000+00:00,11,linux,72.031874,85.322267,86.397208
2020-03-23 17:54:46.956000+00:00,11,linux,34.625169,2.750782,51.012280
2020-03-24 01:13:02.608000+00:00,11,linux,63.849504,53.506142,69.549750
2020-03-24 08:31:18.260000+00:00,11,linux,28.244368,78.966694,40.742563
2020-03-24 15:49:33.913000+00:00,11,linux,95.323246,47.470774,68.003333
2020-03-24 23:07:49.565000+00:00,11,linux,86.179592,21.568282,54.339696


<a id="frames-tsdb-read-conditional"></a>
#### Conditional Read

The following example demonstrates how to use a query filter to conditionally read only a subset of the data from a TSDB table.
This is done by setting the value of the `filter` parameter to a [platform filter expression](https://www.iguazio.com/docs/latest-release/reference/expressions/condition-expression/#filter-expression).

In [15]:
# Read over-time aggregates with a 1-day aggregation step for all metric
# samples in the table with the `os` label "linux" and the `node` label 11.
df = client.read(backend="tsdb", table=tsdb_table, aggregators="count,sum",
                 step="1d", start="0", filter="os=='linux' and node=='11'",
                 multi_index=True)
display(df)

,,,count(cpu),count(disk),count(mem),sum(cpu),sum(disk),sum(mem)
time,node,os,,,,,,
2020-03-22 00:00:00+00:00,11,linux,1.0,1.0,1.0,71.245621,91.954313,20.934462
2020-03-23 00:00:00+00:00,11,linux,3.0,3.0,3.0,110.525082,157.433106,160.797495
2020-03-24 00:00:00+00:00,11,linux,4.0,4.0,4.0,273.596711,201.511892,232.635343
2020-03-25 00:00:00+00:00,11,linux,3.0,3.0,3.0,183.166730,106.699769,167.292110
2020-03-26 00:00:00+00:00,11,linux,3.0,3.0,3.0,71.736371,163.898363,171.576670
2020-03-27 00:00:00+00:00,11,linux,3.0,3.0,3.0,166.073091,78.634711,186.968027
2020-03-28 00:00:00+00:00,11,linux,4.0,4.0,4.0,134.209306,163.710346,127.641747
2020-03-29 00:00:00+00:00,11,linux,3.0,3.0,3.0,171.002976,256.766286,134.784722


<a id="frames-tsdb-delete"></a>
### Delete the TSDB Table

Use the `delete` method of the Frames client with the `tsdb` backend to delete the TSDB table that was used in the previous steps.

In [16]:
client.delete("tsdb", tsdb_table)

<a id="frames-cleanup"></a>
## Cleanup

You can optionally delete any of the directories or files that you created.
See the instructions in the [Creating and Deleting Container Directories](https://www.iguazio.com/docs/latest-release/tutorials/getting-started/containers/#create-delete-container-dirs) tutorial.
For example, the following code uses a local file-system command to delete the entire **&lt;running user&gt;/examples/** directory in the "users" container.
Edit the path, as needed, then remove the comment mark (`#`) and run the code.

In [17]:
#!rm -rf /User/examples/